In [1]:
import pandas as pd

file_path = '건강검진정보.csv'
data = pd.read_csv(file_path, encoding='cp949') 

age_group_counts = data['AGE_GROUP'].value_counts().sort_index()
print(age_group_counts)

#'AGE_GROUP'의 값이 9이상만 존재, 즉 40세 이상의 정보만 나타나 있음

9     162133
10    140254
11    175607
12    134794
13    156036
14     90201
15     73948
16     34120
17     25674
18      7233
Name: AGE_GROUP, dtype: int64


In [2]:
import pandas as pd

file_path = '건강검진정보.csv'
df = pd.read_csv(file_path, encoding='cp949') 

residence_counts = df['SIDO'].value_counts(sort=True, ascending=False)

# 거주지 코드-이름 
residence_mapping = {
    11: '서울특별시',
    42: '강원도',
    26: '부산광역시',
    43: '충청북도',
    27: '대구광역시',
    44: '충청남도',
    28: '인천광역시',
    45: '전라북도',
    29: '광주광역시',
    46: '전라남도',
    30: '대전광역시',
    47: '경상북도',
    31: '울산광역시',
    48: '경상남도',
    36: '세종특별자치시',
    49: '제주특별자치도',
    41: '경기도'
}

residence_counts.index = residence_counts.index.map(residence_mapping)

# 결과
print(residence_counts)

경기도        254167
서울특별시      169859
경상남도        68381
부산광역시       66527
인천광역시       58832
경상북도        53888
대구광역시       47592
충청남도        41825
전라남도        38431
전라북도        37056
충청북도        33171
강원도         30817
대전광역시       28769
광주광역시       27902
울산광역시       24180
제주특별자치도     11889
세종특별자치시      6714
Name: SIDO, dtype: int64


In [5]:
pip install dash

In [7]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

df = pd.read_csv('건강검진정보.csv')
sido_mapping = {
    11: '서울특별시',
    42: '강원도',
    26: '부산광역시',
    43: '충청북도',
    27: '대구광역시',
    44: '충청남도',
    28: '인천광역시',
    45: '전라북도',
    29: '광주광역시',
    46: '전라남도',
    30: '대전광역시',
    47: '경상북도',
    31: '울산광역시',
    48: '경상남도',
    36: '세종특별자치시',
    49: '제주특별자치도',
    41: '경기도'
}

# 초기화
app = dash.Dash(__name__)
app.config['suppress_callback_exceptions'] = True

# 연령대 그래프 생성 
def generate_age_graph(sex, age_group):
    filtered_df = df[(df['SEX'] == sex) & (df['AGE_GROUP'] == age_group)]
    
    # 평균 키, 몸무게 계산
    avg_height = filtered_df['HEIGHT'].mean()
    avg_weight = filtered_df['WEIGHT'].mean()
    
    # 그래프 생성
    fig = go.Figure()
    fig.add_trace(go.Bar(x=['평균 키', '평균 몸무게'], y=[avg_height, avg_weight], name=''))

    return fig

# 거주지 파이 차트 생성 
def generate_residence_pie_chart(age_group):
    age_group_df = df[df['AGE_GROUP'] == age_group]
    residence_counts = age_group_df['SIDO'].map(sido_mapping).value_counts()

    fig = go.Figure(data=[go.Pie(labels=residence_counts.index, values=residence_counts.values)])
    fig.update_layout(title_text=f'거주지 분포')
    return fig

# 대시보드 
app.layout = html.Div([
    html.H1("연령대 별 건강검진정보"),
    
    html.Label("연령대 선택:"),
    dcc.Dropdown(
        id='age-dropdown',
        options=[
            {'label': '40-44세', 'value': 9},
            {'label': '45-49세', 'value': 10},
            {'label': '50-54세', 'value': 11},
            {'label': '55-59세', 'value': 12},
            {'label': '60-64세', 'value': 13},
            {'label': '65-69세', 'value': 14},
            {'label': '70-74세', 'value': 15},
            {'label': '75-79세', 'value': 16},
            {'label': '80-84세', 'value': 17},
            {'label': '85세 이상', 'value': 18}
        ],
        value=9  
    ),
    
    dcc.Graph(id='age-graph'),
    
    dcc.Graph(id='residence-pie-chart')
])

# Callback 함수 
@app.callback(
    [dash.dependencies.Output('age-graph', 'figure'),
     dash.dependencies.Output('residence-pie-chart', 'figure')],
    [dash.dependencies.Input('age-dropdown', 'value')]
)
def update_graph(selected_age_group):
    # 남자 
    male_fig = generate_age_graph(sex=1, age_group=selected_age_group)
    # 여자
    female_fig = generate_age_graph(sex=2, age_group=selected_age_group)

    fig = make_subplots(rows=1, cols=2, subplot_titles=['남자', '여자'])
    fig.add_trace(male_fig['data'][0], row=1, col=1)
    fig.add_trace(female_fig['data'][0], row=1, col=2)

    fig.update_layout(title_text=f'남자와 여자의 평균 키 및 몸무게', title_x=0.5)

    # 거주지 파이 차트
    residence_pie_chart = generate_residence_pie_chart(age_group=selected_age_group)

    return fig, residence_pie_chart

# 캐시 없이 실행
if __name__ == '__main__':
    app.run_server(port=8051, debug=True)
